# GameNet (Hartford et al 2016)

## Session

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import string as string
from __future__ import division
sess = tf.InteractiveSession()

## Data

In [2]:
import_csv = pd.read_csv('gamesmxn.csv')

### Combine same games

In [ ]:
for i in range(1, import_csv.shape[0]):
    if pd.DataFrame(import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]).values.any():
        repeatmat = import_csv['matrixrow'][0:i][import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]].index[0]
        choicesum = np.matrix(import_csv['choicerow'][repeatmat]) + np.matrix(import_csv['choicerow'][i])
        choicesum = string.replace(str(choicesum),'[','')
        choicesum = string.replace(str(choicesum),']]','')
        choicesum = string.replace(str(choicesum),']\n',';')
        import_csv.set_value(repeatmat, 'choicerow', choicesum)
        import_csv.drop([i], inplace=True)

### Select games to use

In [3]:
tmp = np.empty((import_csv.shape[0]))
tmp[:] = np.NAN
for i in range(import_csv.shape[0]):
    if (import_csv['shape'][i] == '3 3' 
        and import_csv['symmetric'][i] == 1 
        and import_csv['paper'][i] != 'stahlwilson1995'):
        tmp[i] = i
index = tmp[~np.isnan(tmp)]

In [4]:
inputs_row = np.zeros((index.shape[0], 3, 3, 2))
inputs_col = np.zeros((index.shape[0], 3, 3, 2))
target_row = np.zeros((index.shape[0], 3, 1))
for j in range(index.shape[0]):
    Ur = np.matrix(import_csv['matrixrow'][int(index[j])])
    Ur = (Ur-np.mean(Ur))/np.std(Ur)
    Uc = np.transpose(Ur)
    ar = np.matrix(import_csv['choicerow'][int(index[j])])
    if ar.shape[1] == 2:
        ar = ar[:,0]/ar[:,1]
    inputs_row[j, :, :, 0] = Ur
    inputs_row[j, :, :, 1] = Uc
    inputs_col[j, :, :, 0] = Uc
    inputs_col[j, :, :, 1] = Ur
    target_row[j] = ar

### Training and test sets 

In [5]:
shuffle = np.random.permutation(range(inputs_row.shape[0]))
index_train = shuffle[0:inputs_row.shape[0]//5*4] 
index_tests = shuffle[inputs_row.shape[0]//5*4:inputs_row.shape[0]]
inputs_row_train = inputs_row[index_train]
inputs_col_train = inputs_col[index_train]
target_row_train = target_row[index_train]
inputs_row_tests = inputs_row[index_tests]
inputs_col_tests = inputs_col[index_tests]
target_row_tests = target_row[index_tests]

## Model

### Parameters

#### Weights and bias

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and pooling

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [8]:
def rw_pool(x,c):
    x_max = tf.reduce_max(x, axis=2)
    x_til = tf.tile(x_max,[1,3,1])
    x_sha = tf.reshape(x_til,[-1,3,3,c])
    return tf.transpose(x_sha, perm=[0,2,1,3])

def cw_pool(x,c):
    x_max = tf.reduce_max(x, axis=1)
    x_til = tf.tile(x_max,[1,3,1])
    return tf.reshape(x_til,[-1,3,3,c])

### Input and target

In [9]:
x_row = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
x_col = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 3, 1])

### Hidden layer 1 (row player)

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [10]:
x_pool1 = tf.concat(3, [x_row, rw_pool(x_row, 2), cw_pool(x_row, 2)])

In [11]:
W_conv1 = weight_variable([1, 1, 6, 50])
b_conv1 = bias_variable([50])
h_conv1 = tf.nn.relu(conv2d(x_pool1, W_conv1) + b_conv1)

### Hidden layer 2 (row player)

In [12]:
x_pool2 = tf.concat(3, [h_conv1, rw_pool(h_conv1, 50), cw_pool(h_conv1, 50)])

In [13]:
W_conv2 = weight_variable([1, 1, 150, 50])
b_conv2 = bias_variable([50])
h_conv2 = tf.nn.relu(conv2d(x_pool2, W_conv2) + b_conv2)

In [14]:
keep_prob = tf.placeholder(tf.float32)
h_conv2_drop = tf.nn.dropout(h_conv2, keep_prob)

<div style="text-align: right"> <h3> Hidden layer 1 (col player) </h3> </div>

In [15]:
x_pool1_col = tf.concat(3, [x_col, rw_pool(x_col, 2), cw_pool(x_col, 2)])

In [16]:
W_conv1_col = weight_variable([1, 1, 6, 50])
b_conv1_col = bias_variable([50])
h_conv1_col = tf.nn.relu(conv2d(x_pool1_col, W_conv1_col) + b_conv1_col)

<div style="text-align: right"> <h3> Hidden layer 2 (col player) </h3> </div>

In [17]:
x_pool2_col = tf.concat(3, [h_conv1_col, rw_pool(h_conv1_col, 50), cw_pool(h_conv1_col, 50)])

In [18]:
W_conv2_col = weight_variable([1, 1, 150, 50])
b_conv2_col = bias_variable([50])
h_conv2_col = tf.nn.relu(conv2d(x_pool2_col, W_conv2_col) + b_conv2_col)

In [19]:
keep_prob_col = tf.placeholder(tf.float32)
h_conv2_col_drop = tf.nn.dropout(h_conv2_col, keep_prob_col)

<div style="text-align: right"> <h3> Action response layer 0 (col player) </h3> </div>

In [20]:
W_rwsm0 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_col_drop, W_rwsm0)

In [21]:
ar_rwsm0_col = tf.reduce_sum(h_conv2_col_drop, axis=2)
ar_sfmx0_col = tf.nn.softmax(ar_rwsm0_col, dim=1)
ar_sfmx0_col = tf.slice(ar_sfmx0_col, [0, 0, 0], [-1, 3, 1])

### Action response layer 1 (row player response to col player)

In [22]:
W_rdsm1 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_drop, W_rdsm1)

In [23]:
ar_rdsm1 = tf.reduce_sum(h_rdsm1, axis=3)
ar_dtpt1 = tf.matmul(ar_rdsm1, ar_sfmx0_col)

In [24]:
y = tf.nn.softmax(ar_dtpt1, dim=1)

### Cost function

In [25]:
beta = 0.01
regularizer = (tf.nn.l2_loss(W_conv1) 
               + tf.nn.l2_loss(W_conv2) 
               + tf.nn.l2_loss(W_conv1_col) 
               + tf.nn.l2_loss(W_conv2_col)
               + tf.nn.l2_loss(W_rwsm0) 
               + tf.nn.l2_loss(W_rdsm1))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + beta * regularizer, reduction_indices=[1]))

### Optimization

In [26]:
train_step = tf.train.AdamOptimizer(0.0002,0.9,0.999,1e-8).minimize(cross_entropy)

## Train

In [27]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [28]:
for i in range(5000):
    if i%5 == 0:
        shuffle = np.random.permutation(range(inputs_row_train.shape[0]))
    inputs_row_train_batch = inputs_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    inputs_col_train_batch = inputs_col_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    target_row_train_batch = target_row_train[shuffle[shuffle.shape[0]//5*(i%5):shuffle.shape[0]//5*(i%5+1)]]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
                x_row: inputs_row_train_batch,
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch, 
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        train_NLL = cross_entropy.eval(feed_dict={
                x_row: inputs_row_train_batch, 
                x_col: inputs_col_train_batch,
                y_: target_row_train_batch,
                keep_prob: 1.0,
                keep_prob_col: 1.0})
        print("step %d, train accuracy %g, train NLL %g"%(i, train_accuracy, train_NLL))
    train_step.run(feed_dict={x_row: inputs_row_train_batch, 
                              x_col: inputs_col_train_batch, 
                              y_: target_row_train_batch, 
                              keep_prob: 0.8,
                              keep_prob_col: 0.8})

step 0, train accuracy 0.333333, train NLL -1.26428
step 100, train accuracy 0.8, train NLL -2.32925
step 200, train accuracy 0.8, train NLL -3.47916
step 300, train accuracy 0.866667, train NLL -5.03307
step 400, train accuracy 0.866667, train NLL -6.82405
step 500, train accuracy 1, train NLL -9.19693
step 600, train accuracy 0.933333, train NLL -11.846
step 700, train accuracy 0.866667, train NLL -14.9005
step 800, train accuracy 0.933333, train NLL -18.7021
step 900, train accuracy 0.866667, train NLL -22.6127
step 1000, train accuracy 1, train NLL -27.0295
step 1100, train accuracy 1, train NLL -31.909
step 1200, train accuracy 0.866667, train NLL -37.1212
step 1300, train accuracy 1, train NLL -42.8654
step 1400, train accuracy 0.933333, train NLL -48.9672
step 1500, train accuracy 0.8, train NLL -55.4236
step 1600, train accuracy 1, train NLL -62.5476
step 1700, train accuracy 1, train NLL -69.712
step 1800, train accuracy 1, train NLL -77.377
step 1900, train accuracy 1, train 

## Test

In [29]:
test_accuracy = accuracy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
test_NLL = cross_entropy.eval(feed_dict={
        x_row: inputs_row_tests,
        x_col: inputs_col_tests,
        y_: target_row_tests, 
        keep_prob: 1.0,
        keep_prob_col: 1.0})
print("test accuracy %g, test NLL %g"%(test_accuracy, test_NLL))

test accuracy 0.9, test NLL -496.158


In [30]:
pd.set_option('display.max_rows', None)
compare_y = y.eval(feed_dict={
                    x_row: inputs_row_tests, 
                    x_col: inputs_col_tests, 
                    y_: target_row_tests, 
                    keep_prob: 1.0, 
                    keep_prob_col: 1.0})
compare_y = compare_y.reshape(compare_y.shape[0], compare_y.shape[1])
compare_y_ = y_.eval(feed_dict={
                        x_row: inputs_row_tests,
                        x_col: inputs_col_tests,
                        y_: target_row_tests, 
                        keep_prob: 1.0,
                        keep_prob_col: 1.0})
compare_y_ = compare_y_.reshape(compare_y_.shape[0], compare_y_.shape[1])
compare = pd.DataFrame(np.concatenate((compare_y, compare_y_), axis=1))
compare.columns = ["y1","y2","y3","y_1","y_2","y_3"]
compare

,y1,y2,y3,y_1,y_2,y_3
0,0.011138,0.985176,3.685282e-03,0.027211,0.945578,0.027211
1,0.590869,0.210588,1.985438e-01,0.650000,0.175000,0.175000
2,0.843376,0.058791,9.783307e-02,0.520000,0.186667,0.293333
3,0.902462,0.023093,7.444493e-02,0.911565,0.027211,0.061224
4,0.567337,0.044450,3.882132e-01,0.480000,0.013333,0.506667
5,0.432438,0.166710,4.008521e-01,0.531915,0.297872,0.170213
6,0.013056,0.986944,1.614471e-08,0.765957,0.234043,0.000000
7,0.000104,0.049494,9.504025e-01,0.013605,0.074830,0.911565
8,0.843376,0.058791,9.783307e-02,0.500000,0.172414,0.327586
9,0.589330,0.085806,3.248640e-01,0.653061,0.088435,0.258503
